Nhóm Sinh Viên Báo Cáo
Họ tên: Nguyễn An Khương
MSSV: 21424028
Họ tên: Mai Văn Long
MSSV: 21424031
Họ tên: Phan Văn Tình
MSSV: 21424057

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

In [ ]:
def edge_detection(image_path, blur_ksize=7, sobel_ksize=1, skipping_threshold=30):
    """
    image_path: link to image
    blur_ksize: kernel size parameter for Gaussian Blurry
    sobel_ksize: size of the extended Sobel kernel; it must be 1, 3, 5, or 7.
    skipping_threshold: ignore weakly edge
    """
    # read image
    img = cv2.imread(image_path)
    
    # convert BGR to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gaussian = cv2.GaussianBlur(gray, (blur_ksize, blur_ksize), 0)
    
    # sobel algorthm use cv2.CV_64F
    sobelx64f = cv2.Sobel(img_gaussian, cv2.CV_64F, 1, 0, ksize=sobel_ksize)
    abs_sobel64f = np.absolute(sobelx64f)
    img_sobelx = np.uint8(abs_sobel64f)

    sobely64f = cv2.Sobel(img_gaussian, cv2.CV_64F, 1, 0, ksize=sobel_ksize)
    abs_sobel64f = np.absolute(sobely64f)
    img_sobely = np.uint8(abs_sobel64f)
    
    # calculate magnitude
    img_sobel = (img_sobelx + img_sobely)/2
    
    # ignore weakly pixel
    for i in range(img_sobel.shape[0]):
        for j in range(img_sobel.shape[1]):
            if img_sobel[i][j] < skipping_threshold:
                img_sobel[i][j] = 0
            else:
                img_sobel[i][j] = 255
    return img_sobel

In [ ]:
#Hiển thị hình ảnh pnm
def show_pnm(file_name):
  with open(file_name, 'r') as f:
    vals = f.read().split()
  w = int(vals[1])
  h = int(vals[2])
  if vals[0] == 'P2': # Grayscale
    pixels = np.array(vals[4:], dtype=np.uint8).reshape((h, w))
  elif vals[0] == 'P3': # RGB
    pixels = np.array(vals[4:], dtype=np.uint8).reshape((h, w, 3))
  else:
    return None
  
  return Image.fromarray(pixels)

In [ ]:
# Đoạn code Python để hiển thị file chứa ma trận ở notebook
# (một dòng của ma trận ứng với một dòng trong file, 
# các phần tử trên mỗi dòng được cách nhau bởi khoảng trắng) 
# Ví dụ để hiển thị file "pixel_importance.txt", ở dòng cuối của code cell
# bạn gõ: show_matrix('pixel_important.txt')
def show_matrix(matrix_file):
  m = np.loadtxt(matrix_file)
  m = m + m.min()
  m = m / m.max()
  m = m * 255
  m = m.astype(np.uint8)
  return Image.fromarray(m)

In [ ]:
show_pnm("in.pnm")

In [ ]:
!nvcc seamcarving_v1.cu -o DA

In [ ]:
!./DA in.pnm 300 512

In [ ]:
show_pnm("300_host.pnm")

# 1. Mô tả ứng dụng

- Input: một tấm ảnh (đồ án này: ảnh RGB)

In [ ]:
show_pnm("in.pnm")

- Output: tấm ảnh được thay đổi kích thước mà không làm biến dạng các
đối tượng quan trọng (đồ án này: tấm ảnh được thu hẹp chiều rộng lại)

In [ ]:
show_pnm("300_host.pnm")

- Ý nghĩa thực tế của ứng dụng: một use case là ta có thể muốn có các
phiên bản với các kích thước khác nhau của cùng một tấm ảnh để hiển
thị trên các thiết bị khác nhau (máy tính, điện thoại, …)
- Ứng dụng này có cần tăng tốc không?
 + Cần, ứng dụng này chạy chậm khi cài đặt tuần tự


# 2. Cài đặt tuần tự

#### 1. Chuyển ảnh sang grayscale

In [ ]:
# Đọc ảnh PNM
image = Image.open("grb_to_gray_in.pnm")
image

#### 2. Tính độ quan trọng của mỗi pixel

- Thực hiện convolution giữa ảnh grayscale với bộ lọc x-Sobe(1)

| 1 | 0 | -1 |
|:-------|:------:|-------:|
| 2 | 0 | -2 |
| 1 | 0 | -1 |

| (1 * f(i-1,j-1)) | (0 * f(i-1,j)) | (-1 * f(i-1,j+1)) |
|:-------|:------:|-------:|
| (2 * f(i,j-1)) | (0 * f(i,j)) | (-2 * f(i,j+1)) |
| (1 * f(i+1,j-1)) | (0 * f(i+1,j)) | (-1 * f(i+1,j+1)) |

- Thực hiện convolution giữa ảnh grayscale với bộ lọc y-Sobe(2)


| 1 | 2 | 1 |
|:-------|:------:|-------:|
| 2 | 0 | 0 |
| -1 | -2 | -1 |

Độ quan trọng của một pixel = + |kết quả tương ứng của (1)| + |kết quả tương ứng của (2)|


#### 3. Tìm seam ít quan trọng nhất từ độ quan trọng của mỗi pixel

- Bắt đầu từ height - 1 chúng ta sẽ tính tổng pixel hiện tại với 3 pixel ở hàng height + 1 (trái - dưới - phải) nếu tổng pixel là nhỏ nhất thì ta sẽ lấy giá trị đó làm pixel

<img src="1.3.1.png">
<img src="1.3.2.png">

#### 4. Xóa seam ít quan trọng nhất